# Proyek Sistem Rekomendasi Game

Nama: MUHAMMAD AGUSRIANSYAH

Email: mc224d5y1338@student.devacademy.id

ID Dicoding: mc224d5y1338

## Tahap 0: Persiapan (Setup)
Di bagian ini, akan menginstal dan mengimpor semua library yang dibutuhkan untuk keseluruhan proyek.

In [1]:
# Menginstal library yang diperlukan
!pip install kagglehub -q

# Mengimpor library
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import kagglehub
from kagglehub import KaggleDatasetAdapter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

print("Semua library berhasil diimpor!")

Semua library berhasil diimpor!


## Tahap 1: Memuat Dataset dan Memahami Data (Data Understanding)

Pada tahap ini, akan memuat dataset Steam Store Games dari Kaggle, memahami struktur data, melihat jumlah baris dan kolom, serta menampilkan beberapa sampel data. Dataset ini berisi informasi berbagai game yang tersedia di platform Steam, termasuk nama game, genre, tag, harga, dan tanggal rilisnya.

Sumber dataset: [Steam Store Games - Kaggle](https://www.kaggle.com/datasets/nikdavis/steam-store-games)


In [2]:
# Tahap 1: Load dataset langsung dari Kaggle menggunakan KaggleHub
from kagglehub import KaggleDatasetAdapter
import kagglehub

# Set path ke file CSV di dalam dataset
file_path = "steam.csv"  # pastikan sesuai dengan nama file di Kaggle

# Load dataset
df = kagglehub.load_dataset(
    KaggleDatasetAdapter.PANDAS,
    "nikdavis/steam-store-games",
    file_path
)

# Cek ukuran data dan tampilkan 5 baris pertama
print("Jumlah baris dan kolom:", df.shape)
df.head()


<ipython-input-2-04b5840af9c9>:9: DeprecationWarning: load_dataset is deprecated and will be removed in a future version.
  df = kagglehub.load_dataset(


Jumlah baris dan kolom: (27075, 18)


,appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price
0,10,Counter-Strike,2000-11-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,124534,3339,17612,317,10000000-20000000,7.19
1,20,Team Fortress Classic,1999-04-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,3318,633,277,62,5000000-10000000,3.99
2,30,Day of Defeat,2003-05-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Valve Anti-Cheat enabled,Action,FPS;World War II;Multiplayer,0,3416,398,187,34,5000000-10000000,3.99
3,40,Deathmatch Classic,2001-06-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,1273,267,258,184,5000000-10000000,3.99
4,50,Half-Life: Opposing Force,1999-11-01,1,Gearbox Software,Valve,windows;mac;linux,0,Single-player;Multi-player;Valve Anti-Cheat en...,Action,FPS;Action;Sci-fi,0,5250,288,624,415,5000000-10000000,3.99


### Struktur Data

Berikut adalah penjelasan beberapa kolom penting dalam dataset Steam Store Games:

- `appid` : ID unik untuk masing-masing game di Steam.
- `name` : Nama game.
- `release_date` : Tanggal rilis game.
- `english` : Indikator apakah game mendukung bahasa Inggris (1: ya, 0: tidak).
- `developer` : Nama pengembang game.
- `publisher` : Nama penerbit game.
- `platforms` : Platform yang didukung (misalnya: Windows, Mac, Linux).
- `required_age` : Batas usia minimum untuk memainkan game.
- `categories` : Kategori fitur game seperti Single-player, Multi-player, dll.
- `genres` : Genre utama game (misalnya: Action, Adventure).
- `steamspy_tags` : Tag tambahan terkait fitur dan konten (mirip genre, lebih granular).
- `achievements` : Jumlah pencapaian (achievements) yang tersedia.
- `positive_ratings` : Jumlah ulasan positif dari pemain.
- `negative_ratings` : Jumlah ulasan negatif dari pemain.
- `average_playtime` : Rata-rata waktu bermain (dalam menit).
- `median_playtime` : Waktu bermain median (dalam menit).
- `owners` : Perkiraan jumlah pemilik game (rentang).
- `price` : Harga game (dalam USD).

Selanjutnya, akan mengecek apakah terdapat nilai kosong (missing value) atau anomali pada data sebelum dilakukan pemodelan.


In [3]:
# Cek missing values
print("Jumlah missing value per kolom:")
print(df.isnull().sum())

Jumlah missing value per kolom:
appid                0
name                 0
release_date         0
english              0
developer            1
publisher           14
platforms            0
required_age         0
categories           0
genres               0
steamspy_tags        0
achievements         0
positive_ratings     0
negative_ratings     0
average_playtime     0
median_playtime      0
owners               0
price                0
dtype: int64


In [4]:
# Cek tipe data
print("\nTipe data masing-masing kolom:")
print(df.dtypes)


Tipe data masing-masing kolom:
appid                 int64
name                 object
release_date         object
english               int64
developer            object
publisher            object
platforms            object
required_age          int64
categories           object
genres               object
steamspy_tags        object
achievements          int64
positive_ratings      int64
negative_ratings      int64
average_playtime      int64
median_playtime       int64
owners               object
price               float64
dtype: object


## Tahap 2: Data Preparation

Setelah memahami struktur data, langkah berikutnya adalah menyiapkan data untuk pemodelan sistem rekomendasi. Tahapan ini meliputi:

- Menghapus baris dengan nilai kosong pada kolom penting (`name`, `genres`, `steamspy_tags`).
- Mengubah format `release_date` menjadi tipe datetime.
- Menggabungkan kolom `genres`, `steamspy_tags`, dan `categories` menjadi satu kolom `content` untuk digunakan sebagai representasi fitur dalam pendekatan Content-Based Filtering.
- Melakukan pengambilan sampel data (sampling) secara acak sebanyak 5.000 baris. Langkah ini diambil untuk mengatasi keterbatasan sumber daya komputasi (RAM) dan memastikan proses modeling dapat berjalan lancar tanpa kegagalan sistem.

In [5]:
# 1. Hapus baris yang tidak memiliki nilai penting (name, genres, tags)
df = df.dropna(subset=['name', 'genres', 'steamspy_tags'])

In [6]:
# 2. Ubah release_date menjadi format datetime
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')

In [7]:
# 3. Buat kolom baru "content" dari genres + tags + categories
df['content'] = (
    df['genres'].fillna('') + ' ' +
    df['steamspy_tags'].fillna('') + ' ' +
    df['categories'].fillna('')
)

In [8]:
# 4. Ambil sampel data (5000 game) agar proses lebih ringan
df_final = df.sample(5000, random_state=42).reset_index(drop=True)

In [9]:
# Cek hasil gabungan
df_final[['name', 'content']].head()

,name,content
0,Sparky's Hunt,Indie Indie;Horror Single-player
1,Endzeit,Action;Early Access Early Access;Action;Violen...
2,Richard & Alice,Adventure;Indie Adventure;Indie;Point & Click ...
3,The Journey Down: Chapter One,Adventure;Indie Adventure;Point & Click;Indie ...
4,In The Long Run The Game,Action;Adventure;Indie;Simulation;Strategy;Ear...


## Tahap 3: Modeling - Content-Based Filtering

Sistem rekomendasi pada proyek ini menggunakan pendekatan Content-Based Filtering, yaitu mencari game yang mirip berdasarkan kemiripan kontennya (genre, kategori, dan tag).

Langkah-langkah yang dilakukan adalah:

1. Mengubah teks `content` menjadi representasi vektor menggunakan TF-IDF Vectorizer, tetapi di batasi fitur TF-IDF ke 5000 kata paling penting.
2. Menghitung kesamaan antar game menggunakan Cosine Similarity.
3. Membuat fungsi `recommend(game_name)` untuk menampilkan top-N game yang mirip dengan game input.


In [10]:
# 1. TF-IDF Vectorization
tfidf = TfidfVectorizer(stop_words='english', max_features=5000)
tfidf_matrix = tfidf.fit_transform(df_final['content'])

In [11]:
# 2. Cosine Similarity antar game
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [12]:
# 3. Buat indeks untuk lookup nama game
indices = pd.Series(df_final.index, index=df_final['name']).drop_duplicates()

In [13]:
# 4. Fungsi rekomendasi
def recommend(game_name, top_n=10):
    # Cek apakah game ada di dalam data sampel kita
    if game_name not in indices:
        return f"Game '{game_name}' tidak ditemukan di dalam data sampel."

    idx = indices[game_name]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:top_n+1]
    game_indices = [i[0] for i in sim_scores]

    return df_final[['name', 'genres', 'steamspy_tags', 'price']].iloc[game_indices]


## Tahap 4: Evaluation

Pada tahap ini, kita akan melakukan evaluasi terhadap model Content-Based Filtering yang telah dibuat. Metrik yang digunakan adalah **Precision@K**.

### Precision@K
Precision@K merupakan metrik yang mengukur proporsi item yang relevan dari sejumlah K item teratas yang direkomendasikan.  Dalam konteks proyek ini, sebuah game dianggap "relevan" jika memiliki setidaknya satu genre yang sama dengan game input.

Formula dari metrik ini adalah:

$$ \text{Precision@K} = \frac{\text{Jumlah item relevan dalam top-K}}{\text{K}} $$

Sebagai contoh, jika merekomendasikan 10 game (K=10) dan 8 di antaranya memiliki genre yang sama dengan game input, maka presisi model adalah 80%.

Berikut adalah hasil pengujian presisi pada salah satu game:

In [14]:
def calculate_precision_at_k(input_game_name, recommendations, k=10):
    # Ambil genre dari game input
    input_game_genres = set(df_final[df_final['name'] == input_game_name]['genres'].iloc[0].split(';'))

    if not recommendations.empty:
        relevant_items = 0
        # Loop sebanyak k rekomendasi teratas
        for idx, row in recommendations.head(k).iterrows():
            recommended_genres = set(row['genres'].split(';'))
            # Cek apakah ada irisan (intersection) genre
            if input_game_genres.intersection(recommended_genres):
                relevant_items += 1

        return relevant_items / k
    return 0

In [15]:
# --- CONTOH PENGGUNAAN ---

# 1. Pilih satu game secara acak dari dataset sampel Anda untuk diuji
test_game_name = df_final['name'].sample(1).iloc[0]
print(f"Game yang dipilih secara acak untuk diuji: {test_game_name}")

Game yang dipilih secara acak untuk diuji: Dragon's Dogma: Dark Arisen


In [16]:
# 2. Dapatkan rekomendasinya
top_10_recommendations = recommend(test_game_name, top_n=10)

In [17]:
# 3. Hitung presisinya
precision = calculate_precision_at_k(test_game_name, top_10_recommendations, k=10)

In [19]:
print(f"Hasil rekomendasi untuk '{test_game_name}':")
print(top_10_recommendations)

Hasil rekomendasi untuk 'Dragon's Dogma: Dark Arisen':
                                             name                      genres  \
1042                         Way of the Samurai 3            Action;Adventure   
4918  The Elder Scrolls V: Skyrim Special Edition                         RPG   
4791                                         ELEX        Action;Adventure;RPG   
802                 Middle-earth™: Shadow of War™        Action;Adventure;RPG   
4829                    Kingdom Come: Deliverance        Action;Adventure;RPG   
382                              Sunset Overdrive            Action;Adventure   
4006         Oceanhorn: Monster of Uncharted Seas  Action;Adventure;Indie;RPG   
1558             Mount & Blade: With Fire & Sword                  Action;RPG   
3856                               Drifting Lands            Action;Indie;RPG   
3048                                 Phoning Home      Action;Adventure;Indie   

                                 steamspy_tags  price

In [20]:
print(f"\nPrecision@10 untuk game '{test_game_name}' adalah: {precision:.0%}")


Precision@10 untuk game 'Dragon's Dogma: Dark Arisen' adalah: 100%


### Interpretasi Hasil Evaluasi

Dari hasil pengujian yang dijalankan pada game yang dipilih secara acak, yaitu 'Dragon's Dogma: Dark Arisen', didapatkan nilai **Precision@10 sebesar 100%**.

Nilai ini menunjukkan bahwa dari 10 game teratas yang direkomendasikan oleh sistem, seluruhnya (10 dari 10) memiliki setidaknya satu genre yang sama dengan 'Dragon's Dogma: Dark Arisen'.

Hasil presisi yang sangat tinggi ini mengindikasikan bahwa model *Content-Based Filtering* yang telah dibangun sangat efektif dalam menemukan dan merekomendasikan game dengan kemiripan konten yang relevan, sesuai dengan tujuan yang telah ditetapkan.